In [ ]:
#Inversed filters

In [ ]:
#!/usr/bin/env python3
import os
import glob
import sys
import time

import cv2
import torch
import numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR) – increase if too strict
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    # Crop to rail ROI
    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # Vectorized color match
    targets_bgr = np.array([(b,g,r) for r,g,b in target_colors_rgb], np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)

    combined_roi = mask_roi & color_mask_roi

    # Connected-component filter
    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Returns:
      - out        : original + red tint on rails + neon-green on unfiltered rails
      - rail_mask  : boolean mask of all rails
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        return img_bgr.copy(), np.zeros((H, W), bool)

    # union of rail segments
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # compute filtered (original neon-green) mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB, TOLERANCE,
        MIN_REGION_SIZE, MIN_REGION_HEIGHT
    )

    # inverted mask = rails minus the filtered zones
    inverted_mask = rail_mask & ~filtered_mask

    # compose visualization
    out = img_bgr.copy()

    # red tint for rails
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)

    # neon-green on the *inverse* region
    out[inverted_mask] = (0, 255, 0)

    return out, rail_mask

# =======================
# Main: run & step through
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, _ = process_frame(img)

        # show original | inverted neon-green
        combo = np.hstack((img, vis))
        cv2.imshow("Original (Left) | Inverted Rails (Right)", combo)

        # SPACE → next, 'q' → quit
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs


2025-08-05 01:56:26.815 Python[40138:28838161] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-05 01:56:26.815 Python[40138:28838161] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [ ]:
#!/usr/bin/env python3
import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR) – increase if too strict
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# Centerline settings
N_LINES           = 3
MIN_GAP_FRAC      = 0.15      # min horizontal separation between lines (fraction of width)
BAND_FRAC         = 0.04      # half-width (as fraction of image width) for local centroid band
Y_TOP_FRAC        = 0.25      # start tracing from this fraction downwards (perspective)
BOTTOM_FOCUS_FRAC = 0.60      # use bottom 60% to score column density
THICKNESS         = 6
PURPLE            = (255, 0, 255)  # BGR

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # RGB -> BGR
    targets_bgr = np.array([(b,g,r) for r,g,b in target_colors_rgb], np.float32)
    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)

    combined_roi = mask_roi & color_mask_roi

    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Column-peak picking (no SciPy)
# =======================
def pick_top_columns(mask: np.ndarray, n: int, min_gap: int) -> list[int]:
    """
    Greedily pick up to n columns with highest vertical density, enforcing min_gap separation.
    Uses bottom portion of the image to score columns (more reliable).
    """
    H, W = mask.shape
    y_start = int(H * (1 - BOTTOM_FOCUS_FRAC))
    dens = mask[y_start:, :].sum(axis=0).astype(np.float32)

    order = np.argsort(-dens)  # descending
    chosen = []
    for x in order:
        if dens[x] <= 0:
            break
        if all(abs(int(x) - int(c)) >= min_gap for c in chosen):
            chosen.append(int(x))
            if len(chosen) == n:
                break
    chosen.sort()
    return chosen

def trace_centerline(mask: np.ndarray, x0: int, band_w: int,
                     y_top: int, step: int = 4, smooth: float = 0.6) -> np.ndarray:
    """
    Follow dense green region from bottom to y_top by taking the centroid within [x0-band_w, x0+band_w].
    Returns Nx1x2 array of int points for cv2.polylines.
    """
    H, W = mask.shape
    pts = []
    x_prev = x0

    for y in range(H-1, y_top-1, -step):
        xl = max(0, x_prev - band_w)
        xr = min(W, x_prev + band_w + 1)
        row = mask[y, xl:xr]
        if row.any():
            xs = np.nonzero(row)[0] + xl
            x_c = xs.mean()
            x_prev = int(round(smooth * x_prev + (1.0 - smooth) * x_c))
        # if no pixels, keep previous x_prev (continues straight)
        pts.append([x_prev, y])

    if len(pts) >= 2:
        return np.array(pts, dtype=np.int32).reshape(-1, 1, 2)
    return np.empty((0, 1, 2), dtype=np.int32)

def draw_three_centerlines(out_img: np.ndarray, green_mask: np.ndarray) -> None:
    """
    Find 3 dense green centerlines and draw them in purple on out_img in-place.
    """
    H, W = green_mask.shape
    min_gap = max(12, int(W * MIN_GAP_FRAC))
    band_w  = max(6,  int(W * BAND_FRAC))
    y_top   = int(H * Y_TOP_FRAC)

    xs = pick_top_columns(green_mask, N_LINES, min_gap)
    for x0 in xs:
        poly = trace_centerline(green_mask, x0, band_w, y_top)
        if poly.size > 0:
            cv2.polylines(out_img, [poly], isClosed=False, color=PURPLE, thickness=THICKNESS, lineType=cv2.LINE_AA)

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Returns:
      - out        : original + rails red tint + inverted green + 3 purple centerlines
      - rail_mask  : boolean mask of all rails
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        return img_bgr.copy(), np.zeros((H, W), bool)

    # union of rail segments
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # compute filtered (original neon-green) mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB, TOLERANCE,
        MIN_REGION_SIZE, MIN_REGION_HEIGHT
    )

    # inverted mask = rails minus the filtered zones (this is your green background)
    inverted_mask = rail_mask & ~filtered_mask

    # compose visualization
    out = img_bgr.copy()

    # red tint for rails
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)

    # neon-green on the *inverse* region
    out[inverted_mask] = (0, 255, 0)

    # --- draw 3 purple lines in the densest green regions (no overlap) ---
    draw_three_centerlines(out, inverted_mask)

    return out, rail_mask

# =======================
# Main: run & step through
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, _ = process_frame(img)

        combo = np.hstack((img, vis))
        cv2.imshow("Original (Left) | Inverted Rails + 3 Purple Lines (Right)", combo)

        # SPACE → next, 'q' → quit
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs


2025-08-05 01:30:39.590 Python[38613:28807275] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-05 01:30:39.590 Python[38613:28807275] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [ ]:
#Heatmap straight 

In [1]:
#!/usr/bin/env python3
import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR) – increase if too strict
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(rbg[2], rbg[1], rbg[0]) for rbg in target_colors_rgb],
                           dtype=np.float32)  # (K,3)

    img_f = img_roi.astype(np.float32)                                  # (h,w,3)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]        # (h,w,K,3)
    dist2 = np.sum(diff * diff, axis=-1)                                 # (h,w,K)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)   # (h,w)

    combined_roi = mask_roi & color_mask_roi

    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Heatmap (red-dominance vs green)
# =======================
def red_vs_green_heatmap(red_mask: np.ndarray, green_mask: np.ndarray,
                         blur_ksize: int = 51) -> np.ndarray:
    """
    Build a heatmap where high values (red) indicate regions with more *red* pixels than green,
    and low values (blue) indicate fewer red pixels. Uses a large blur for local density.
    """
    H, W = red_mask.shape
    # convert to float densities then smooth
    k = (blur_ksize, blur_ksize)
    red_d   = cv2.blur(red_mask.astype(np.float32),   k)
    green_d = cv2.blur(green_mask.astype(np.float32), k)
    diff = red_d - green_d  # >0 => red-dominant, <0 => green-dominant

    # normalize to 0..255 with zero centered around ~128 using symmetric scaling
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)  # 0..1 with 0.5 at tie
    norm_u8 = np.clip(norm * 255.0, 0, 255).astype(np.uint8)

    heat = cv2.applyColorMap(norm_u8, cv2.COLORMAP_JET)  # blue -> red
    return heat

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns:
      - out_bgr     : original with red rail mask + neon green filtered regions
      - rail_mask   : boolean rail mask
      - heatmap_bgr : red-vs-green heatmap (blue=less red, red=more red)
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        empty = np.zeros((H, W), bool)
        heat  = red_vs_green_heatmap(empty, empty)
        return img_bgr, empty, heat

    # Union of rail masks
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # Apply fast color/size filter inside rails (these are the green regions)
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        target_colors_rgb=TARGET_COLORS_RGB,
        tolerance=TOLERANCE,
        min_region_size=MIN_REGION_SIZE,
        min_region_height=MIN_REGION_HEIGHT
    )

    # red area = rails but not filtered
    red_area   = rail_mask & (~filtered_mask)
    green_area = filtered_mask

    # Compose visualization
    out = img_bgr.copy()
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)       # red tint for all rails
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)
    out[filtered_mask] = (0, 255, 0)       # neon green for filtered regions

    # Heatmap below: red-dominance vs green
    heat = red_vs_green_heatmap(red_area, green_area)

    return out, rail_mask, heat

# =======================
# Run over a folder: show side-by-side with heatmap underneath
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        vis, _rail, heat = process_frame(img)

        # Top row: original | rails+filtered
        top = np.hstack((img, vis))
        # Bottom: heatmap stretched to same width
        heat_resized = cv2.resize(heat, (top.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)

        display_img = np.vstack((top, heat_resized))
        cv2.imshow("Top: Original (Left) | Rails+Filtered (Right)  |  Bottom: Red-vs-Green Heatmap", display_img)

        # Wait for SPACE (next) or 'q' (quit)
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:        # SPACE
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs


2025-08-05 02:03:22.587 Python[40446:28844547] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-05 02:03:22.587 Python[40446:28844547] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


KeyboardInterrupt: 

In [ ]:
#Heatmap overlay with rail ends

In [ ]:
#!/usr/bin/env python3
import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0      # color distance (BGR) – increase if too strict
MIN_REGION_SIZE   = 30        # connected-component area in px
MIN_REGION_HEIGHT = 150       # connected-component bbox height in px

# ---- Heatmap→triangle picking params ----
HEAT_BLUR_KSIZE     = 51            # local averaging
RED_SCORE_THRESH    = 220           # 0..255; "dark red" threshold
EXCLUDE_TOP_PX      = 400            # ignore top Y pixels
EXCLUDE_BOTTOM_PX   = 120           # ignore bottom X pixels (from bottom up)
MIN_DARK_RED_AREA   = 1200          # min area (px) for a dark-red blob
TRI_SIZE_PX         = 18            # triangle size
PURPLE              = (255, 0, 255) # BGR

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Returns boolean mask of rail pixels that:
      (1) color-match any target color within tolerance, AND
      (2) lie inside rail_mask, AND
      (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    # RGB -> BGR targets
    targets_bgr = np.array([(rbg[2], rbg[1], rbg[0]) for rbg in target_colors_rgb],
                           dtype=np.float32)

    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)

    combined_roi = mask_roi & color_mask_roi

    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Heatmap (red-dominance vs green) + score map
# =======================
def red_vs_green_heatmap(red_mask: np.ndarray, green_mask: np.ndarray,
                         blur_ksize: int = 51) -> tuple[np.ndarray, np.ndarray]:
    """
    Build a heatmap where high values (red) indicate regions with more *red* pixels than green,
    and low values (blue) indicate fewer red pixels. Also returns the normalized score (0..255).
    """
    # convert to float densities then smooth
    k = (blur_ksize, blur_ksize)
    red_d   = cv2.blur(red_mask.astype(np.float32),   k)
    green_d = cv2.blur(green_mask.astype(np.float32), k)
    diff = red_d - green_d  # >0 => red-dominant, <0 => green-dominant

    # normalize to 0..255 with zero centered around ~128 using symmetric scaling
    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)  # 0..1 with 0.5 at tie
    norm_u8 = np.clip(norm * 255.0, 0, 255).astype(np.uint8)

    heat = cv2.applyColorMap(norm_u8, cv2.COLORMAP_JET)  # blue -> red
    return heat, norm_u8

# =======================
# Draw a small triangle marker with apex at (x, y)
# =======================
def draw_purple_triangle(img: np.ndarray, x: int, y: int, size: int = TRI_SIZE_PX):
    h = int(size * 1.2)
    pts = np.array([
        [x, y],                    # apex
        [x - size, y + h],         # base left
        [x + size, y + h],         # base right
    ], dtype=np.int32)
    cv2.fillConvexPoly(img, pts, PURPLE)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns:
      - out_bgr     : original with rail tint + green regions + purple triangles at rail ends
      - rail_mask   : boolean rail mask
      - heatmap_bgr : red-vs-green heatmap (blue=more green, red=more red)
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        empty = np.zeros((H, W), bool)
        heat, _ = red_vs_green_heatmap(empty, empty, blur_ksize=HEAT_BLUR_KSIZE)
        return img_bgr.copy(), empty, heat

    # union of rail segments
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # compute filtered (green) mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB, TOLERANCE,
        MIN_REGION_SIZE, MIN_REGION_HEIGHT
    )

    # red area = rails minus the filtered zones
    red_area   = rail_mask & (~filtered_mask)
    green_area = filtered_mask

    # Compose visualization
    orig_marked = img_bgr.copy()             # triangles go on ORIGINAL
    out = img_bgr.copy()                      # rails viz on the right panel
    overlay = out.copy()
    overlay[rail_mask] = (0, 0, 255)
    out = cv2.addWeighted(overlay, ALPHA, out, 1 - ALPHA, 0)
    out[filtered_mask] = (0, 255, 0)

    # Heatmap + score (0..255, 255 = most red-dominant)
    heat, score = red_vs_green_heatmap(red_area, green_area, blur_ksize=HEAT_BLUR_KSIZE)

    # ---- pick dark-red blobs within Y-range and mark their uppermost point ----
    score_mask = (score >= RED_SCORE_THRESH).astype(np.uint8)

    # limit vertical search band
    top = max(0, EXCLUDE_TOP_PX)
    bot = max(0, EXCLUDE_BOTTOM_PX)
    score_mask[:top, :] = 0
    if bot > 0:
        score_mask[H - bot:, :] = 0

    # remove tiny speckle
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9))
    score_mask = cv2.morphologyEx(score_mask, cv2.MORPH_OPEN, kernel, iterations=1)

    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(score_mask, 8)
    for lbl in range(1, n_labels):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area < MIN_DARK_RED_AREA:
            continue

        # uppermost y in this blob
        mask_lbl = (labels == lbl)
        ys, xs = np.where(mask_lbl)
        y_min = int(ys.min())

        # x at the top-most row for stability (mean of that row's xs)
        xs_top = xs[ys == y_min]
        x_mid = int(np.clip(int(np.round(xs_top.mean())), 0, W - 1))

        # draw triangle on the ORIGINAL image
        draw_purple_triangle(orig_marked, x_mid, y_min, TRI_SIZE_PX)

    return (orig_marked, rail_mask, heat), out  # return both left(original+triangles) and right(viz), plus heat

# =======================
# Run over a folder: show side-by-side with heatmap underneath
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        (orig_with_tri, _rail, heat), rails_viz = process_frame(img)

        # Top row: ORIGINAL(with triangles) | rails+filtered viz
        top = np.hstack((orig_with_tri, rails_viz))

        # Bottom: heatmap resized to match width
        heat_resized = cv2.resize(heat, (top.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)

        display_img = np.vstack((top, heat_resized))
        cv2.imshow("Top: Original + Rail-End Triangles (Left) | Rails+Filtered (Right)  |  Bottom: Red-vs-Green Heatmap", display_img)

        # SPACE → next, 'q' → quit
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs


2025-08-05 02:14:05.905 Python[40926:28855408] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-05 02:14:05.905 Python[40926:28855408] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [ ]:
#Working on %

In [ ]:
#!/usr/bin/env python3
import os, glob, sys, time
import cv2, torch, numpy as np
from ultralytics import YOLO

# =======================
# Config
# =======================
home     = os.path.expanduser("~")
weights  = f"{home}/models/jakes-loped/jakes-finder-mk1/1/weights.pt"
RAIL_ID  = 9

IMG_SIZE = 512
CONF, IOU = 0.30, 0.45
ALPHA    = 0.40  # red tint on rail mask

# Color/size filter (RGB targets; converted to BGR internally)
TARGET_COLORS_RGB = [(119,104,67), (81,42,45)]
TOLERANCE         = 20.0
MIN_REGION_SIZE   = 30
MIN_REGION_HEIGHT = 150

# ---- Heatmap→triangle picking params ----
HEAT_BLUR_KSIZE       = 51            # local averaging window
RED_SCORE_THRESH      = 220           # 0..255; "dark red" threshold
EXCLUDE_TOP_FRAC      = 0.40          # exclude top 10% of the image (before analysis)
EXCLUDE_BOTTOM_FRAC   = 0.15          # exclude bottom 15% of the image (before analysis)
MIN_DARK_RED_AREA     = 1200          # min area (px) for a dark-red blob
TRI_SIZE_PX           = 18            # triangle size
PURPLE                = (255, 0, 255) # BGR

# =======================
# Device / precision
# =======================
if torch.cuda.is_available():
    device, half = 0, True
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device, half = "mps", False
else:
    device, half = "cpu", False

# =======================
# Model (load, fuse, warmup)
# =======================
model = YOLO(weights)
try:
    model.fuse()
except Exception:
    pass

_dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
_ = model.predict(
    _dummy, task="segment", imgsz=IMG_SIZE, device=device,
    conf=CONF, iou=IOU, classes=[RAIL_ID],
    verbose=False, half=half
)

# =======================
# FAST color+size filter (vectorized + ROI-cropped)
# =======================
def highlight_rails_mask_only_fast(
    img_bgr: np.ndarray,
    rail_mask: np.ndarray,
    target_colors_rgb: list[tuple[int,int,int]],
    tolerance: float = 30.0,
    min_region_size: int = 50,
    min_region_height: int = 150
) -> np.ndarray:
    """
    Boolean mask of rail pixels that (1) color-match within tolerance,
    (2) lie inside rail_mask, and (3) pass connected-component size/height filters.
    """
    H, W = img_bgr.shape[:2]
    filtered_full = np.zeros((H, W), dtype=bool)
    if not rail_mask.any():
        return filtered_full

    ys, xs = np.where(rail_mask)
    y0, y1 = ys.min(), ys.max() + 1
    x0, x1 = xs.min(), xs.max() + 1

    img_roi  = img_bgr[y0:y1, x0:x1]
    mask_roi = rail_mask[y0:y1, x0:x1]

    targets_bgr = np.array([(rbg[2], rbg[1], rbg[0]) for rbg in target_colors_rgb],
                           dtype=np.float32)

    img_f = img_roi.astype(np.float32)
    diff  = img_f[:, :, None, :] - targets_bgr[None, None, :, :]
    dist2 = np.sum(diff * diff, axis=-1)
    color_mask_roi = np.any(dist2 <= (tolerance * tolerance), axis=-1)

    combined_roi = mask_roi & color_mask_roi

    comp_u8 = combined_roi.astype(np.uint8)
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(comp_u8, 8)

    filtered_roi = np.zeros_like(combined_roi)
    for lbl in range(1, n_labels):
        area   = stats[lbl, cv2.CC_STAT_AREA]
        height = stats[lbl, cv2.CC_STAT_HEIGHT]
        if area >= min_region_size and height >= min_region_height:
            filtered_roi[labels == lbl] = True

    filtered_full[y0:y1, x0:x1] = filtered_roi
    return filtered_full

# =======================
# Heatmap (red-dominance vs green) + score map
# =======================
def red_vs_green_heatmap(red_mask: np.ndarray, green_mask: np.ndarray,
                         blur_ksize: int = 51) -> tuple[np.ndarray, np.ndarray]:
    """
    Heatmap (JET) and normalized score 0..255 where higher means more *red* than green.
    """
    k = (blur_ksize, blur_ksize)
    red_d   = cv2.blur(red_mask.astype(np.float32),   k)
    green_d = cv2.blur(green_mask.astype(np.float32), k)
    diff = red_d - green_d  # >0 => red-dominant, <0 => green-dominant

    amax = float(np.max(np.abs(diff))) + 1e-6
    norm = (diff / (2.0 * amax) + 0.5)  # 0..1 with 0.5 at tie
    norm_u8 = np.clip(norm * 255.0, 0, 255).astype(np.uint8)

    heat = cv2.applyColorMap(norm_u8, cv2.COLORMAP_JET)
    return heat, norm_u8

# =======================
# Draw a small triangle marker with apex at (x, y)
# =======================
def draw_purple_triangle(img: np.ndarray, x: int, y: int, size: int = TRI_SIZE_PX):
    h = int(size * 1.2)
    pts = np.array([
        [x, y],                    # apex
        [x - size, y + h],         # base left
        [x + size, y + h],         # base right
    ], dtype=np.int32)
    cv2.fillConvexPoly(img, pts, PURPLE)
    cv2.polylines(img, [pts.reshape(-1,1,2)], True, (0,0,0), 1, cv2.LINE_AA)

# =======================
# Per-frame pipeline
# =======================
def process_frame(img_bgr: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns:
      - orig_with_tri : ORIGINAL image with purple triangles at uppermost points of dark-red heat regions
      - rail_mask     : boolean rail mask
      - heatmap_bgr   : red-vs-green heatmap (blue=more green, red=more red)
    """
    res = model.predict(
        img_bgr, task="segment", imgsz=IMG_SIZE, device=device,
        conf=CONF, iou=IOU, classes=[RAIL_ID],
        max_det=20, verbose=False, half=half
    )[0]

    H, W = img_bgr.shape[:2]
    if res.masks is None:
        empty = np.zeros((H, W), bool)
        heat, _ = red_vs_green_heatmap(empty, empty, blur_ksize=HEAT_BLUR_KSIZE)
        return img_bgr.copy(), empty, heat

    # union of rail segments
    m = res.masks.data
    union = (m.sum(dim=0) > 0).float().cpu().numpy()
    if union.shape != (H, W):
        union = cv2.resize(union, (W, H), interpolation=cv2.INTER_NEAREST)
    rail_mask = union.astype(bool)

    # compute filtered (green) mask
    filtered_mask = highlight_rails_mask_only_fast(
        img_bgr, rail_mask,
        TARGET_COLORS_RGB, TOLERANCE,
        MIN_REGION_SIZE, MIN_REGION_HEIGHT
    )

    # red area = rails minus the filtered zones
    red_area   = rail_mask & (~filtered_mask)
    green_area = filtered_mask

    # Heatmap + score (0..255, 255 = most red-dominant)
    heat, score = red_vs_green_heatmap(red_area, green_area, blur_ksize=HEAT_BLUR_KSIZE)

    # ---- percentage-based exclusion applied BEFORE analysis for triangle plotting ----
    top_rows = int(H * max(0.0, min(1.0, EXCLUDE_TOP_FRAC)))
    bot_rows = int(H * max(0.0, min(1.0, EXCLUDE_BOTTOM_FRAC)))
    score[:top_rows, :] = 0
    if bot_rows > 0:
        score[H - bot_rows:, :] = 0

    # threshold dark-red regions
    score_mask = (score >= RED_SCORE_THRESH).astype(np.uint8)

    # denoise small speckle
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 9))
    score_mask = cv2.morphologyEx(score_mask, cv2.MORPH_OPEN, kernel, iterations=1)

    # find blobs and mark the uppermost point per blob
    orig_marked = img_bgr.copy()
    n_labels, labels, stats, _ = cv2.connectedComponentsWithStats(score_mask, 8)
    for lbl in range(1, n_labels):
        area = stats[lbl, cv2.CC_STAT_AREA]
        if area < MIN_DARK_RED_AREA:
            continue
        mask_lbl = (labels == lbl)
        ys, xs = np.where(mask_lbl)
        y_min = int(ys.min())
        xs_top = xs[ys == y_min]
        x_mid = int(np.clip(int(np.round(xs_top.mean())), 0, W - 1))
        draw_purple_triangle(orig_marked, x_mid, y_min, TRI_SIZE_PX)

    return orig_marked, rail_mask, heat

# =======================
# Run over a folder: show side-by-side with heatmap underneath
# =======================
if __name__ == "__main__":
    image_dir = f"{home}/SubwaySurfers/train_screenshots"
    paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))
    if not paths:
        print("No frames found in", image_dir)
        sys.exit(1)

    for p in paths:
        img = cv2.imread(p)
        if img is None:
            continue

        orig_with_tri, _rail, heat = process_frame(img)

        # For reference, also render rails+filtered (right pane)
        # (Optional: remove this block if you only want original + heatmap)
        overlay = img.copy()
        res_rails = model.predict(
            img, task="segment", imgsz=IMG_SIZE, device=device,
            conf=CONF, iou=IOU, classes=[RAIL_ID],
            max_det=20, verbose=False, half=half
        )[0]
        if res_rails.masks is not None:
            m = res_rails.masks.data
            union = (m.sum(dim=0) > 0).float().cpu().numpy()
            if union.shape != img.shape[:2]:
                union = cv2.resize(union, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
            rail_mask_viz = union.astype(bool)
            overlay2 = img.copy()
            overlay2[rail_mask_viz] = (0, 0, 255)
            right = cv2.addWeighted(overlay2, ALPHA, img, 1 - ALPHA, 0)
        else:
            right = img.copy()

        top = np.hstack((orig_with_tri, right))
        heat_resized = cv2.resize(heat, (top.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)
        display_img = np.vstack((top, heat_resized))

        cv2.imshow("Top: Original + Purple Rail-End Triangles (Left) | Rails Viz (Right)  |  Bottom: Red-vs-Green Heatmap", display_img)

        # SPACE → next, 'q' → quit
        while True:
            key = cv2.waitKey(0) & 0xFF
            if key == 32:
                break
            elif key == ord('q'):
                cv2.destroyAllWindows()
                sys.exit(0)

        cv2.destroyAllWindows()


YOLO11n-seg summary (fused): 113 layers, 2,836,908 parameters, 0 gradients, 10.2 GFLOPs


2025-08-05 02:16:45.803 Python[41068:28858725] +[IMKClient subclass]: chose IMKClient_Legacy
2025-08-05 02:16:45.803 Python[41068:28858725] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
